In [8]:
import pandas as pd
from transformers import pipeline
import torch
from datasets import Dataset
import numpy as np

In [9]:
# read the data from resources/data and present them
test = pd.read_csv("../resources/data/test.csv")
train = pd.read_csv("../resources/data/training.csv")
train.head()

,id,sentence,label
0,0,Those 2 drinks are part of the HK culture and ...,negative
1,1,I was told by the repair company that was doin...,negative
2,2,It is there to give them a good time .,neutral
3,3,Like leafing through an album of photos accomp...,negative
4,4,Johnny was a talker and liked to have fun.,positive


In [10]:
# use gpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [11]:
CANDIDATES = ["positive", "negative", "neutral"]
ds = train#.head(10_000).copy()        # keep your original slice
sentences = ds["sentence"].tolist()

roberta_clf = pipeline(
    task="sentiment-analysis",
    model="cardiffnlp/twitter-roberta-base-sentiment-latest",
    tokenizer="cardiffnlp/twitter-roberta-base-sentiment-latest",
    device=0,
    batch_size=64,
    padding=True,
    max_length=512,                    
    truncation=True
)
roberta_preds = roberta_clf(sentences, truncation=True)
ds["pred"] = [p["label"].lower() for p in roberta_preds]

ds.head()


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


,id,sentence,label,pred
0,0,Those 2 drinks are part of the HK culture and ...,negative,negative
1,1,I was told by the repair company that was doin...,negative,negative
2,2,It is there to give them a good time .,neutral,positive
3,3,Like leafing through an album of photos accomp...,negative,negative
4,4,Johnny was a talker and liked to have fun.,positive,positive


In [12]:

_LABEL2NUM = {
    "negative": -1, -1: -1,
    "neutral":   0,  0:  0,
    "positive":  1,  1:  1,
}
def _to_num(x):

    if isinstance(x, str):
        x = x.strip().lower()
    return _LABEL2NUM[x]


def sentiment_score(y_true, y_pred):
    """
    Implements   L = 0.5 * ( 2 − 1/n Σ |y_i − ŷ_i| )
    Returns a float in [0, 1].
    """
    y_t = np.fromiter((_to_num(t) for t in y_true), dtype=np.int8)
    y_p = np.fromiter((_to_num(p) for p in y_pred), dtype=np.int8)
    mean_abs_diff = np.abs(y_t - y_p).mean()
    return 0.5 * (2.0 - mean_abs_diff)




results = sentiment_score(ds["label"], ds["pred"])
print("Sentiment score: ", results)


Sentiment score:  0.8292261280938715


# Roberta is the chosen one